In [37]:
import pandas as pd
import requests

# Step 1: Parse the .sql file (unchanged)
def extract_sql_commands_and_questions(file_path):
    sql_commands = []
    questions = []
    current_question = None
    current_command = []

    with open(file_path, "r") as file:
        for line in file:
            stripped_line = line.strip()
            
            if stripped_line.startswith("--"):
                # If we encounter a new question, save any pending command first
                if current_command and current_question:
                    sql_commands.append((current_question, ' '.join(current_command)))
                    current_command = []
                
                # Extract the new question
                current_question = stripped_line.replace("--", "", 1).strip()
                questions.append(current_question)
            
            elif stripped_line:  # Non-empty, non-comment line
                current_command.append(stripped_line)
            
            else:  # Empty line - end of current command
                if current_command and current_question:
                    sql_commands.append((current_question, ' '.join(current_command)))
                    current_command = []
    
        # Add the last command if file doesn't end with empty line
        if current_command and current_question:
            sql_commands.append((current_question, ' '.join(current_command)))

    return sql_commands, questions

# Step 2: Execute "SQL-like" commands on XLSX data (unchanged)
def execute_xlsx_commands(xlsx_file_path, sql_commands):
    # Load the Excel file into a Pandas DataFrame
    data = pd.ExcelFile(xlsx_file_path)
    results = []

    for question, command in sql_commands:
        try:
            # Example: Interpret SQL commands as DataFrame operations
            if "SELECT" in command.upper():
                # Extract sheet name and query details (simplified example)
                sheet_name, query = command.replace("SELECT", "").strip().split(" FROM ")
                df = data.parse(sheet_name.strip())  # Load the specified sheet into a DataFrame
                
                # Perform filtering or column selection based on the query
                # For simplicity, assume query specifies columns (e.g., "column1, column2")
                columns = [col.strip() for col in query.split(",")]
                result = df[columns].to_dict(orient="records")  # Convert filtered DataFrame to list of dictionaries
            else:
                result = f"Unsupported command: {command}"
            
            results.append((question, command, result))
        except Exception as e:
            results.append((question, command, f"Error: {str(e)}"))

    return results

# Step 3: Use Generative AI to analyze results (modified for DeepSeek)
def generate_hypotheses(results):
    prompt = "Analyze these results and generate hypotheses:\n\n"
    for question, command, result in results:
        prompt += f"Question: {question}\nData: {result}\n\n"

    try:
        response = requests.post(
            "http://localhost:11434/api/chat",  # Changed from /api/generate to /api/chat
            json={
                "model": "qwen2.5-coder:1.5b-base",
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
            }
        )
        response.raise_for_status()
        return response.json()["message"]["content"]
    except Exception as e:
        return f"API Error: {str(e)}"

# Main workflow (unchanged)
def main():
    sql_file_path = r"C:\Users\LucySouza\projects\internal\pipeline_detectives\data\earlier_analyses\analise_negocios.sql"  # Path to your .sql file
    xlsx_file_path = r"C:\Users\LucySouza\projects\internal\pipeline_detectives\data\raw_data\olist tabelas.xlsx"  # Path to your Excel file

    # Step 1: Extract SQL commands and questions
    sql_commands, questions = extract_sql_commands_and_questions(sql_file_path)

    # Step 2: Execute commands on Excel
    results = execute_xlsx_commands(xlsx_file_path, sql_commands)

    # Step 3: Generate hypotheses using Generative AI
    hypotheses = generate_hypotheses(results)

    # Output the results
    print("AI-Generated Hypotheses:")
    print(hypotheses)

In [24]:
sql_file_path = r"C:\Users\LucySouza\projects\internal\pipeline_detectives\data\earlier_analyses\analise_negocios.sql"  # Path to your .sql file
xlsx_file_path = r"C:\Users\LucySouza\projects\internal\pipeline_detectives\data\raw_data\olist tabelas.xlsx"  # Path to your Excel file

In [38]:
sql_commands, questions = extract_sql_commands_and_questions(sql_file_path)

In [43]:
print(questions)

['1. Qual a categoria que possui o produto com o maior nÃºmero de dias entre a primeira compra da categoria e a sua data limite de entrega?', '2. Qual o nome da categoria com o maior nÃºmero de pedidos realizados no banco de dados?', '3. Qual a categoria com maior soma dos preÃ§os de produtos?', '4. Qual o cÃ³digo do produto mais caro da categoria agro indÃºstria & comercio?', '5. Qual a ordem correta das 3 categorias com os produtos mais caros?', '6. Qual o valor dos produtos mais caros das categorias: bebes, flores e seguros e serviÃ§os, respectivamente:', '7.Quantos pedidos possuem um Ãºnico comprador, 3 produtos e o pagamento foi dividido em 10 parcelas?', '8. Quantos pedidos foram parcelados em mais de 10 vezes ?', '9. Quantos clientes avaliaram o pedido com 5 estrelas?', '10. Quantos clientes avaliaram o pedido com 4 estrelas?', '11. Quantos clientes avaliaram o pedido com 3 estrelas?', '12. Quantos clientes avaliaram o pedido com 2 estrelas?', '13. Quantos clientes avaliaram o p

In [42]:
print(sql_commands[1][0])

2. Qual o nome da categoria com o maior nÃºmero de pedidos realizados no banco de dados?


In [44]:
results = execute_xlsx_commands(xlsx_file_path, sql_commands)
print(results)

[('1. Qual a categoria que possui o produto com o maior nÃºmero de dias entre a primeira compra da categoria e a sua data limite de entrega?', 'WITH FirstPurchase AS ( SELECT p.product_category_name, MIN(o.order_purchase_timestamp) AS first_purchase_date FROM ORDERS o JOIN ORDER_ITEMS oi ON o.order_id = oi.order_id JOIN PRODUCTS p ON oi.product_id = p.product_id GROUP BY p.product_category_name ), DeliveryDates AS ( SELECT p.product_category_name, o.order_id, o.order_estimated_delivery_date, fp.first_purchase_date, (julianday(o.order_estimated_delivery_date) - julianday(fp.first_purchase_date)) AS days_diff FROM ORDERS o JOIN ORDER_ITEMS oi ON o.order_id = oi.order_id JOIN PRODUCTS p ON oi.product_id = p.product_id JOIN FirstPurchase fp ON p.product_category_name = fp.product_category_name ) SELECT product_category_name, MAX(days_diff) AS max_days_diff FROM DeliveryDates GROUP BY product_category_name ORDER BY max_days_diff DESC LIMIT 1;', 'Error: too many values to unpack (expected 2)

In [23]:
# Run the workflow
if __name__ == "__main__":
    main()

AI-Generated Hypotheses:
Question: no ranking de produtos mais caros dessa categoria?
Data: Error: not enough values to unpack (expected 2, got 1)

Question: None
Data: Unsupported command: p.product_id,

Question: None
Data: Unsupported command: SUM(oi.price) AS total_value

Question: None
Data: Unsupported command: FROM

Question: None
Data: Unsupported command: PRODUCTS p

Question: None
Data: Unsupported command: JOIN

Question: None
Data: Unsupported command: ORDER_ITEMS oi ON p.product_id = oi.product_id

Question: None
Data: Unsupported command: WHERE

Question: None
Data: Unsupported command: p.product_category_name = 'artes'

Question: None
Data: Unsupported command: GROUP BY

Question: None
Data: Unsupported command: p.product_id

Question: None
Data: Unsupported command: ORDER BY

Question: None
Data: Unsupported command: total_value DESC;

Question: mais caros da categoria brinquedos.
Data: Error: not enough values to unpack (expected 2, got 1)

Question: None
Data: Unsuppo